# Miscellaneos experiemnts
A place to try stuff out...

In [ ]:
import sys
sys.path += ['../Src']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Ensuring dataframes have identical gene sets

In [ ]:
def read_pair_of_expressions_and_intersect(file_1, file_2):
    """Read expression data from both files returning a pair of dataframes having only genes which
    are present in both files"""
    df_1 = pd.read_csv(file_1, sep='\t', index_col=0)
    df_2 = pd.read_csv(file_2, sep='\t', index_col=0)
    
    genes_1 = set(df_1.index)
    genes_2 = set(df_2.index)
    intersection_genes = genes_1.intersection(genes_2)
    
    intersection_df = pd.DataFrame()
    intersection_df['GeneENSG'] = list(intersection_genes)
    intersection_df.set_index('GeneENSG', inplace=True)
    
    df_1_pruned = pd.merge(df_1, intersection_df, how='inner', on='GeneENSG')
    df_2_pruned = pd.merge(df_2, intersection_df, how='inner', on='GeneENSG')
    df_1_pruned.sort_index(inplace=True)
    df_2_pruned.sort_index(inplace=True)
    
    print(len(genes_1), len(genes_2), len(df_1_pruned))
    
    assert len(df_1_pruned) == len(df_2_pruned)
    assert all(df_1_pruned.index.values == df_2_pruned.index.values)
    
    return df_1_pruned, df_2_pruned

In [ ]:
expression_filename_1 = '../Data/AOCS_Protein/AOCS_Protein_Expression.tsv'
expression_filename_2 = '../Data/TCGA_OV_VST/TCGA_OV_VST_Expression.tsv'

df_1, df_2 = read_pair_of_expressions_and_intersect(expression_filename_1, expression_filename_2)

In [ ]:
df_2

## Computing H from W and X

In [ ]:
from sklearn.linear_model import LinearRegression
from factorizer_wrappers import ICA_Factorizer

In [ ]:
X = np.random.randn(50, 10) + 1.0

In [ ]:
facto = ICA_Factorizer(n_components=3,tol=0.01)
facto.fit(X)

In [ ]:
W = facto.get_W()
W.shape

In [ ]:
H = facto.get_H()
H.shape

In [ ]:
np.linalg.pinv(W).shape

In [ ]:
H_by_pinv = np.dot(np.linalg.pinv(W), X)

In [ ]:
H_by_pinv

In [ ]:
H

# Bar plots

In [ ]:
n = 6
data = np.random.randn(3, n)

In [ ]:
X = np.arange(n)
plt.bar(X + 0.00, data[0], color = 'b', width = 0.25)
plt.bar(X + 0.25, data[1], color = 'g', width = 0.25)
plt.bar(X + 0.50, data[2], color = 'r', width = 0.25)

## Experimenting with Heatmaps

In [ ]:
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# Data set
url = 'https://python-graph-gallery.com/wp-content/uploads/mtcars.csv'
df = pd.read_csv(url)
df = df.set_index('model')
df


In [ ]:
# Prepare a vector of color mapped to the 'cyl' column
# my_palette = dict(zip(df.cyl.unique(), ["orange","yellow","brown"]))
# row_colors = df.cyl.map(my_palette)

def make_colour_col(target):
    viridis = cm.get_cmap('viridis', 20)
    tmin, tmax = min(target), max(target)
    row_colors = [viridis((v-tmin)/(tmax-tmin)) for v in target]
    return row_colors


from matplotlib import cm

target_df = df[[]].copy()
target_df['disp'] = make_colour_col(df['disp'])
target_df['hp'] = make_colour_col(df['hp'])
target_df['gear'] = make_colour_col(df['gear'])
target_df['_'] = [(0,0,0,0)] * len(df)

target_df

In [ ]:
# plot
sns.clustermap(df, metric="correlation", method="single", cmap="Blues", standard_scale=1,
               row_colors=target_df, tree_kws={'color':'r'})
plt.show()
